In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('Data/fashion', one_hot=True)

Extracting Data/fashion/train-images-idx3-ubyte.gz
Extracting Data/fashion/train-labels-idx1-ubyte.gz
Extracting Data/fashion/t10k-images-idx3-ubyte.gz
Extracting Data/fashion/t10k-labels-idx1-ubyte.gz


In [3]:
n_classes = 10
input_size = 784

x = tf.placeholder(tf.float32, shape=[None, input_size])
y = tf.placeholder(tf.float32, shape=[None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [5]:
def summary_variable(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stdev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stdev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [6]:
x_image = tf.reshape(x, [-1,28,28,1])
with tf.name_scope('weights'):
    W_conv1 = weight_variable([7, 7, 1, 100])
    summary_variable(W_conv1)
with tf.name_scope('biases'):
    b_conv1 = bias_variable([100])
    summary_variable(b_conv1)

with tf.name_scope('Conv1'):
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    tf.summary.histogram('activations', h_conv1)
    tf.summary.histogram('max_pool', h_pool1)

with tf.name_scope('weights'):
    W_conv2 = weight_variable([4, 4, 100, 150])
    summary_variable(W_conv2)
with tf.name_scope('biases'):
    b_conv2 = bias_variable([150])
    summary_variable(b_conv2)

with tf.name_scope('Conv2'):
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    tf.summary.histogram('activations', h_conv2)
    tf.summary.histogram('max_pool', h_pool2)

In [7]:
W_conv3 = weight_variable([4, 4, 150, 250])
b_conv3 = bias_variable([250])
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)

W_fc1 = weight_variable([4 * 4 * 250, 300])
b_fc1 = bias_variable([300])
h_pool3_flat = tf.reshape(h_pool3, [-1, 4*4*250])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([300, n_classes])
b_fc2 = bias_variable([n_classes])
y_pred = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [8]:
with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_pred)
    with tf.name_scope('total'):
        cross_entropy = tf.reduce_mean(diff)
tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [9]:
learning_rate = 0.001
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [10]:
sess = tf.InteractiveSession()
log_dir = 'tensorboard-example-weights'
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
val_writer = tf.summary.FileWriter(log_dir + '/val')

In [ ]:
n_steps = 1000
batch_size = 128
dropout = 0.25
evaluate_every = 10
n_val_steps = mnist.test.images.shape[0] // batch_size

In [ ]:
tf.global_variables_initializer().run()
for i in range(n_steps):
    x_batch, y_batch = mnist.train.next_batch(batch_size)
    summary, _, train_acc = sess.run([merged, train_step, accuracy], feed_dict={x: x_batch, y: y_batch, keep_prob: dropout})
    train_writer.add_summary(summary, i)

    if i % evaluate_every == 0:
        val_accs = []
        for j in range(n_val_steps):
            x_batch, y_batch = mnist.test.next_batch(batch_size)
            summary, val_acc = sess.run([merged, accuracy], feed_dict={x: x_batch, y: y_batch, keep_prob: 1.0})
            val_writer.add_summary(summary, i)
            val_accs.append(val_acc)
        print('Step {:04.0f}: train_acc: {:.4f}; val_acc {:.4f}'.format(i, train_acc, sum(val_accs)/len(val_accs)))
train_writer.close()
val_writer.close()

Step 0000: train_acc: 0.1406; val_acc 0.2179
Step 0010: train_acc: 0.1797; val_acc 0.4831
Step 0020: train_acc: 0.3203; val_acc 0.4927
Step 0030: train_acc: 0.2812; val_acc 0.6179
Step 0040: train_acc: 0.2891; val_acc 0.7009
Step 0050: train_acc: 0.3828; val_acc 0.7347
Step 0060: train_acc: 0.4766; val_acc 0.7823
Step 0070: train_acc: 0.4453; val_acc 0.8366
Step 0080: train_acc: 0.4844; val_acc 0.8559
